### moduls install

In [ ]:
!pip install tensorflow==2.9.0
!pip install "numpy<2.0"
!pip install matplotlib seaborn scikit-learn opencv-python-headless

### import moduls

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

### dir setting

In [ ]:
model_dir = 'best_model'
structure_path = os.path.join(model_dir, 'model_structure.json')
weights_path = os.path.join(model_dir, 'model_weights.h5')
config_path = os.path.join(model_dir, 'config.json')

### model loading

In [ ]:
def load_model_with_config(structure_path, weights_path, config_path):
    # structure
    with open(structure_path, 'r') as f:
        model = model_from_json(f.read())

    # weights
    model.load_weights(weights_path)

    # setting info
    with open(config_path, 'r') as f:
        config = json.load(f)
    print("📌 Loaded config:", config)

    # optimizer set
    optimizer = tf.keras.optimizers.get({
        'class_name': config.get('optimizer', 'Adam'),
        'config': {'learning_rate': config.get('learning_rate', 0.001)}
    })

    # model compile
    model.compile(
        optimizer=optimizer,
        loss=config.get('loss_function', 'categorical_crossentropy'),
        metrics=config.get('metrics', ['accuracy'])
    )
    
    return model, config


### input data

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_path = 'test_images/sample.jpg'  # 여기에 새 이미지 경로
img = load_img(img_path, target_size=(224, 224))  # 모델 입력 사이즈에 맞게 수정!
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:

# 전처리 함수
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 학습할 때 rescale=1./255 했던 것과 동일하게 맞춤
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

### run

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# path setting
structure_path = 'best_model/model_structure.json'
weights_path = 'best_model/model_weights.h5'
config_path = 'best_model/config.json'
class_names_path = 'best_model/class_names.json'

# model, class loading
model, config = load_model_with_config(structure_path, weights_path, config_path)

with open(class_names_path, 'r') as f:
    class_names = json.load(f)


In [ ]:
# 후처리 함수
def decode_prediction(pred, class_names):
    class_index = np.argmax(pred, axis=1)[0]
    confidence = np.max(pred)
    return class_names[class_index], confidence

# precition 
def predict_image(image_path, model, class_names):
    img = preprocess_image(image_path)
    pred = model.predict(img)
    pred_class, confidence = decode_prediction(pred, class_names)
    return pred_class, confidence

In [ ]:
# running model
img_path = 'your_image.jpg'  # 예측할 이미지 경로
pred_class, confidence = predict_image(img_path, model, class_names)

print(f"📌 예측 클래스: {pred_class} ({confidence * 100:.2f}%)")